<a href="https://colab.research.google.com/github/gcvasconcelos/unb-classification_political_bias/blob/main/tcc_manifestos_1extracao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Construção da base de treino - Manifesto Project


In [ ]:
import pandas as pd
import numpy as np
import requests
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Construção da base de metadados



### Coletando metadados dos Manifestos do Brasil

In [ ]:
meta_sa_df = pd.read_csv("/content/drive/MyDrive/UnB/TCC/dados_originais/MPDataset_MPDSSA2020b.csv")
meta_sa_df.info()
meta_sa_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Columns: 108 entries, country to id_perm
dtypes: float64(91), int64(9), object(8)
memory usage: 87.9+ KB


,country,countryname,oecdmember,edate,date,party,partyname,partyabbrev,parfam,candidatename,...,per704,per705,per706,rile,planeco,markeco,welfare,intpeace,datasetversion,id_perm
0,150,Argentina,0,14/05/1989,198905,150029,Justicialist Party of Popular Unity,FREJUPO,40,Carlos S. Menem,...,0.0,0.000,2.145,-3.820,8.981,3.418,3.083,0.067,2020b,RM63HS
1,150,Argentina,0,14/05/1989,198905,150040,Center Alliance,NaN,40,Álvaro Alsogaray,...,0.0,0.000,0.000,13.287,0.699,19.580,7.692,4.196,2020b,QT6YB6
2,150,Argentina,0,14/05/1989,198905,150301,Civic Radical Union,UCR,30,Eduardo Angeloz,...,0.0,0.503,2.314,-19.416,4.930,3.018,10.966,2.012,2020b,QCSEOS
3,150,Argentina,0,14/05/1995,199505,150201,Justicialist Party,PJ,40,Carlos S. Menem,...,0.0,0.399,1.118,1.997,3.834,9.425,5.431,1.597,2020b,HGJWJD
4,150,Argentina,0,14/05/1995,199505,150202,Front for a Country in Solidarity,FREPASO,20,José O. Bordón,...,0.0,0.000,0.000,-15.217,8.696,6.522,21.739,0.000,2020b,CPBGUM


In [ ]:
meta_brazil_df = meta_sa_df[meta_sa_df['country'] == 180]
meta_brazil_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 76 to 103
Columns: 108 entries, country to id_perm
dtypes: float64(91), int64(9), object(8)
memory usage: 23.8+ KB


In [ ]:
meta_brazil_df[['party', 'partyname', 'partyabbrev', 'parfam', 'candidatename']]

,party,partyname,partyabbrev,parfam,candidatename
76,180230,Popular Front Brazil,NaN,20,Luiz Inácio Lula da Silva
77,180250,Democratic Labour Party,PDT,20,Leonel de Moura Brizola
78,180309,Party of Brazilian Social Democracy,PSDB,50,Mário Covas Júnior
79,180410,New Brazil Movement,NaN,40,Fernando Affonso Collor de Mello
80,180610,Democratic Social Party,PDS,60,Paulo Salim Maluf
81,180230,A democratic revolution in Brazil,NaN,20,Luiz Inácio Lula da Silva
82,180310,Let’s work Brazil,NaN,60,Fernando Henrique Cardoso
83,180710,Party of the Reconstruction of the National Order,PRONA,70,Enéas Carneiro
84,180230,Union of the People - Change Brazil,NaN,20,Luiz Inácio Lula da Silva
85,180232,Socialist People's Party,PPS,20,Ciro Gomes


In [ ]:
meta_brazil_df['manifest_id'] = meta_brazil_df['party'].astype(str) + '_' + meta_brazil_df['date'].astype(str)
meta_brazil_df[['manifest_id', 'candidatename']].reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,manifest_id,candidatename
0,76,180230_198911,Luiz Inácio Lula da Silva
1,77,180250_198911,Leonel de Moura Brizola
2,78,180309_198911,Mário Covas Júnior
3,79,180410_198911,Fernando Affonso Collor de Mello
4,80,180610_198911,Paulo Salim Maluf
5,81,180230_199410,Luiz Inácio Lula da Silva
6,82,180310_199410,Fernando Henrique Cardoso
7,83,180710_199410,Enéas Carneiro
8,84,180230_199810,Luiz Inácio Lula da Silva
9,85,180232_199810,Ciro Gomes


### Coletando metadados dos Manifestos de Portugal

In [ ]:
meta_global_df = pd.read_csv("/content/drive/MyDrive/UnB/TCC/dados_originais/MPDataset_MPDS2020b.csv")
meta_global_df.info()
meta_global_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4656 entries, 0 to 4655
Columns: 174 entries, country to id_perm
dtypes: float64(157), int64(10), object(7)
memory usage: 6.2+ MB


,country,countryname,oecdmember,eumember,edate,date,party,partyname,partyabbrev,parfam,...,per608_3,per703_1,per703_2,rile,planeco,markeco,welfare,intpeace,datasetversion,id_perm
0,11,Sweden,0,0,17/09/1944,194409,11220,Communist Party of Sweden,SKP,20,...,NaN,NaN,NaN,9.60,1.9,1.900,0.0,1.900,2020b,JN1LZH
1,11,Sweden,0,0,17/09/1944,194409,11320,Social Democratic Labour Party,SAP,30,...,NaN,NaN,NaN,-37.80,3.3,2.200,33.4,5.600,2020b,CMR7F6
2,11,Sweden,0,0,17/09/1944,194409,11420,People’s Party,FP,40,...,NaN,NaN,NaN,9.50,3.2,6.400,14.3,1.600,2020b,Z6OL6C
3,11,Sweden,0,0,17/09/1944,194409,11620,Right Party,NaN,60,...,NaN,NaN,NaN,28.00,1.8,22.800,10.6,0.000,2020b,YMKVN2
4,11,Sweden,0,0,17/09/1944,194409,11810,Agrarian Party,NaN,80,...,NaN,NaN,NaN,23.81,0.0,19.048,0.0,4.762,2020b,U4SCRD


In [ ]:
meta_portugal_df = meta_global_df[meta_global_df['country'] == 35]
meta_portugal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 1780 to 1890
Columns: 174 entries, country to id_perm
dtypes: float64(157), int64(10), object(7)
memory usage: 151.8+ KB


In [ ]:
meta_portugal_df['manifest_id'] = meta_portugal_df['party'].astype(str) + '_' + meta_portugal_df['date'].astype(str)
meta_portugal_df[['manifest_id', 'partyname']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,manifest_id,partyname
1780,35210_197504,Popular Democratic Union
1781,35220_197504,Portuguese Communist Party
1782,35310_197504,Popular Democratic Movement
1783,35311_197504,Socialist Party
1784,35313_197504,Popular Democratic Party
...,...,...
1886,35311_201910,Socialist Party
1887,35313_201910,Social Democratic Party
1888,35410_201910,Liberal Iniciative
1889,35520_201910,Social Democratic Center-Popular Party


### Agregando metadados e salvando base

In [ ]:
meta_df = meta_portugal_df.append(meta_brazil_df)
meta_df.info()
meta_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139 entries, 1780 to 103
Columns: 176 entries, country to candidatename
dtypes: float64(158), int64(9), object(9)
memory usage: 192.2+ KB


,country,countryname,oecdmember,eumember,edate,date,party,partyname,partyabbrev,parfam,...,per703_2,rile,planeco,markeco,welfare,intpeace,datasetversion,id_perm,manifest_id,candidatename
1780,35,Portugal,10,0.0,25/04/1975,197504,35210,Popular Democratic Union,UDP,20,...,NaN,-55.372,0.000,0.000,4.959,0.826,2020b,Y1GQ3J,35210_197504,NaN
1781,35,Portugal,10,0.0,25/04/1975,197504,35220,Portuguese Communist Party,PCP,20,...,NaN,-44.586,0.000,0.000,0.000,0.000,2020b,S7TB52,35220_197504,NaN
1782,35,Portugal,10,0.0,25/04/1975,197504,35310,Popular Democratic Movement,MDP,30,...,NaN,-37.879,2.273,0.758,10.606,0.000,2020b,OA17UI,35310_197504,NaN
1783,35,Portugal,10,0.0,25/04/1975,197504,35311,Socialist Party,PS,30,...,NaN,-58.108,4.054,0.000,0.000,2.703,2020b,FRUEBV,35311_197504,NaN
1784,35,Portugal,10,0.0,25/04/1975,197504,35313,Popular Democratic Party,PPD,60,...,NaN,-1.887,7.547,1.887,20.755,3.774,2020b,PI4RAA,35313_197504,NaN


In [ ]:
set(meta_portugal_df.columns) ^ set(meta_brazil_df.columns)

{'candidatename',
 'eumember',
 'per1011',
 'per1012',
 'per1013',
 'per1014',
 'per1015',
 'per1016',
 'per1021',
 'per1022',
 'per1023',
 'per1024',
 'per1025',
 'per1026',
 'per103',
 'per1031',
 'per1032',
 'per1033',
 'per201',
 'per202',
 'per2021',
 'per2022',
 'per2023',
 'per2031',
 'per2032',
 'per2033',
 'per2041',
 'per3011',
 'per305',
 'per3051',
 'per3052',
 'per3053',
 'per3054',
 'per3055',
 'per4011',
 'per4012',
 'per4013',
 'per4014',
 'per4121',
 'per4122',
 'per4123',
 'per4124',
 'per4131',
 'per4132',
 'per416',
 'per5021',
 'per5031',
 'per5041',
 'per5061',
 'per601',
 'per6011',
 'per6012',
 'per6013',
 'per6014',
 'per602',
 'per605',
 'per606',
 'per6061',
 'per607',
 'per6071',
 'per6072',
 'per608',
 'per6081',
 'per703',
 'per7051',
 'per7052',
 'per7061',
 'per7062'}

In [ ]:
meta_df.to_csv('/content/drive/MyDrive/UnB/TCC/dados_temporarios/manifestos_metadata.csv', index=False)

In [ ]:
meta_df.parfam.value_counts()

20    48
30    28
60    24
50    16
10    13
70     7
40     2
95     1
Name: parfam, dtype: int64

### Coletando metadados de Códigos 

In [ ]:
codes_df = pd.read_csv("/content/drive/MyDrive/UnB/TCC/dados_originais/codebook_categories_MPDS2020a.csv", dtype=str)
codes_df.info()
codes_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   type            143 non-null    object
 1   domain_code     143 non-null    object
 2   domain_name     142 non-null    object
 3   code            143 non-null    object
 4   variable_name   143 non-null    object
 5   title           143 non-null    object
 6   description_md  143 non-null    object
 7   label           143 non-null    object
dtypes: object(8)
memory usage: 9.1+ KB


,type,domain_code,domain_name,code,variable_name,title,description_md,label
0,main,0,NaN,000,peruncod,No other category applies,Share of uncoded sentences (quasi-sentences co...,percent of uncoded quasi-sentences (000)
1,main,1,External Relations,101,per101,Foreign Special Relationships: Positive,Favourable mentions of particular countries wi...,foreign special +
2,main,1,External Relations,102,per102,Foreign Special Relationships: Negative,Negative mentions of particular countries with...,foreign special -
3,main,1,External Relations,103,per103,Anti-Imperialism,Negative references to imperial behaviour and/...,anti-imperalism
4,main,1,External Relations,104,per104,Military: Positive,The importance of external security and defenc...,military +


Criando uma coluna para o código de headers e substituindo a do código de sentenças sem código.

In [ ]:
codes_df.loc[len(codes_df)] = ['main','0','No Domain','NA','peruncod','Uncoded','Share of uncoded sentences','uncoded']
codes_df.loc[len(codes_df)] = ['main','0','No Domain','H','perhead','Headers','Share of headers sentences','header']
codes_df = codes_df.iloc[1:]

In [ ]:
codes_df.to_csv('/content/drive/MyDrive/UnB/TCC/dados_temporarios/manifestos_code_metadata.csv', index=False)

## Construção da base de sentenças

In [ ]:
host = "https://manifesto-project.wzb.eu/api/v1"

params = ""
for id in meta_df['manifest_id'].tolist():
    params = params + f"&keys[]={id}"

r = requests.get(
  f"{host}/texts_and_annotations?version=2020-2{params}&api_key=779c82c5afcdf6fb6fd6262d0bd5a39a"
)
r.url

'https://manifesto-project.wzb.eu/api/v1/texts_and_annotations?version=2020-2&keys[]=35210_197504&keys[]=35220_197504&keys[]=35310_197504&keys[]=35311_197504&keys[]=35313_197504&keys[]=35520_197504&keys[]=35210_197604&keys[]=35220_197604&keys[]=35311_197604&keys[]=35313_197604&keys[]=35520_197604&keys[]=35710_197604&keys[]=35210_197912&keys[]=35220_197912&keys[]=35310_197912&keys[]=35311_197912&keys[]=35313_197912&keys[]=35520_197912&keys[]=35710_197912&keys[]=35210_198010&keys[]=35220_198010&keys[]=35310_198010&keys[]=35311_198010&keys[]=35313_198010&keys[]=35316_198010&keys[]=35317_198010&keys[]=35520_198010&keys[]=35710_198010&keys[]=35110_198304&keys[]=35210_198304&keys[]=35220_198304&keys[]=35310_198304&keys[]=35311_198304&keys[]=35313_198304&keys[]=35520_198304&keys[]=35710_198304&keys[]=35110_198510&keys[]=35210_198510&keys[]=35220_198510&keys[]=35310_198510&keys[]=35311_198510&keys[]=35312_198510&keys[]=35313_198510&keys[]=35520_198510&keys[]=35110_198707&keys[]=35210_198707&ke

In [ ]:
num_manifestos = meta_df['manifest_id'].nunique()
print(f"Número de manifestos total: {num_manifestos}")
print(f"Número de manifestos com sentenças codificadas: {num_manifestos - len(r.json()['missing_items'])}")
r.json()['missing_items']

Número de manifestos total: 139
Número de manifestos com sentenças codificadas: 102


['35520_197504',
 '35520_197604',
 '35313_197912',
 '35520_197912',
 '35710_197912',
 '35311_198010',
 '35313_198010',
 '35316_198010',
 '35317_198010',
 '35520_198010',
 '35710_198010',
 '35310_198304',
 '35520_198304',
 '35110_198510',
 '35520_198510',
 '35312_198707',
 '35520_198707',
 '35229_199110',
 '35520_199110',
 '35951_199110',
 '35229_199510',
 '35520_199510',
 '35229_199910',
 '35520_199910',
 '35211_200203',
 '35229_200203',
 '35520_200203',
 '35220_200502',
 '35520_200502',
 '35110_200909',
 '35211_200909',
 '35220_200909',
 '35311_200909',
 '35313_200909',
 '35520_200909',
 '180250_198911',
 '180610_198911']

In [ ]:
sentences_array = [dict(sentence, **{'manifest_id':corpus['key']}) for corpus in r.json()['items'] for sentence in corpus['items']]
sentences_array[10:15]

[{'cmp_code': '506',
  'eu_code': 'NA',
  'manifest_id': '180110_201010',
  'text': 'Elevação do investimento em educação do setor público para 7% do PIB, priorizando novos investimentos na direção da universalização do acesso à pré-escola e à creche;'},
 {'cmp_code': '506',
  'eu_code': 'NA',
  'manifest_id': '180110_201010',
  'text': 'Eliminação do analfabetismo entre jovens de 15 a 30 anos até 2014'},
 {'cmp_code': '506',
  'eu_code': 'NA',
  'manifest_id': '180110_201010',
  'text': 'e erradicação do analfabetísmo até 2018;'},
 {'cmp_code': '701',
  'eu_code': 'NA',
  'manifest_id': '180110_201010',
  'text': 'Valorização dos professores da rede pública na sua remuneração,'},
 {'cmp_code': '411',
  'eu_code': 'NA',
  'manifest_id': '180110_201010',
  'text': 'acesso universal a computador e internet,'}]

In [ ]:
manifests_df = pd.DataFrame(sentences_array, columns=['manifest_id', 'text', 'cmp_code'])
manifests_df.info()
manifests_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92096 entries, 0 to 92095
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   manifest_id  92096 non-null  object
 1   text         92096 non-null  object
 2   cmp_code     92096 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


,manifest_id,text,cmp_code
0,180110_201010,Agenda por um Brasil Justo e Sustentável,NA
1,180110_201010,Propostas de Marina Silva e do Partido Verde p...,NA
2,180110_201010,As propostas foram elaboradas a partir das Dir...,NA
3,180110_201010,Transparência e ética,NA
4,180110_201010,Não instituicão de qualquer mecanismo de tutel...,201.2


Detecção de manifestos com apenas uma sentença para remoção.

In [ ]:
one_sample = manifests_df.manifest_id.value_counts()
one_sample = one_sample[one_sample == 1]
print(f"Número de manifestos com apenas 1 sentença: {one_sample.size}")
print("Exemplo:")
print(manifests_df.loc[manifests_df.manifest_id == '35220_198510']["cmp_code"])
manifests_df.loc[manifests_df.manifest_id == '35220_198510', 'text'].item()

Número de manifestos com apenas 1 sentença: 48
Exemplo:
57622    NA
Name: cmp_code, dtype: object


'VITÓRIA DA APU PARA SALVAR O PAÍS   AS ELEIÇÕES ANTECIPADAS PARA A ASSEMBLEIA DA REPÚBLICA DE 6 DE OUTUBRO realizam-se porque a firme, corajosa e persistente lufa do povo e o funcionamento dos instituições democráticas interromperam o processo contra-revolucionário, o caminho de destruição, desastre, fome e miséria prosseguido peio Governo PS/PSD. Realizam-se C porque Mono Soares e o seu Governo PS/PSD foram derrotados e demitidos. Porque com a dissolução da Assembleia da Repúblico, foi posto fim à chamada «maior maioria de sempre» que tantos males causou ao País. O PCP APEIA AO POVO PORTUGUÊS PARA QUE NÃO PERCA A GRANDE OPORTUNIDADE DAS ELEIÇÕES DE Ó DE OUTUBRO PARA ALCANÇAR ATRAVÉS DO VOTO UMA REAL VIRAGEM DEMOCRÁTICA NA POLÍTICA PORTUGUESA.  Três grandes  objectivos nas  eleições  de 6 de Outubro  Q PCP APONTA AO POVO PORTUGUÊS TRÊS GRANDES OBJECTIVOS NAS ELEIÇÕES DE Ó DE OUTUBRO:  PRIMEIRO: ATIRAR DEFINITIVAMENTE PARA A RUA (SE O NÃO FOR ANTES) O GOVERNO DE MÁRIO SOARES/PS/PSD, QU

In [ ]:
manifests_df = manifests_df[~manifests_df['manifest_id'].isin(list(one_sample.index))]
manifests_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92048 entries, 0 to 92095
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   manifest_id  92048 non-null  object
 1   text         92048 non-null  object
 2   cmp_code     92048 non-null  object
dtypes: object(3)
memory usage: 2.8+ MB


Padronizando labels das duas bases

In [ ]:
print(manifests_df.cmp_code.unique())
manifests_df.cmp_code = manifests_df.cmp_code.replace('000', 'NA')
manifests_df.cmp_code = manifests_df.cmp_code.replace('H', 'NA')
print(manifests_df.cmp_code.unique())

['NA' '201.2' '202.1' '202.4' '506' '701' '411' '504' '605.1' '501' '000'
 '503' '416.2' '303' '403' '106' '607.3' '502' '301' 'H' '305.1' '606.2'
 '203' '305.2' '408' '103.2' '703.1' '304' '602.1' '601.1' '706' '415'
 '413' '202.2' '305.5' '204' '105' '606.1' '410' '103.1' '406' '101' '108'
 '107' '201.1' '202.3' '409' '412' '414' '404' '407' '305.6' '705' '405'
 '109' '703.2' '607.1' '605.2' '104' '402' '505' '704' '416.1' '302' '401'
 '305.3' '603' '604' '602.2' '110' '702' '507' '607.2' '608.1' '305' '703'
 '606' '202' '416' '608' '201' '605' '102' '103' '607' '601' '601.2'
 '608.2']
['NA' '201.2' '202.1' '202.4' '506' '701' '411' '504' '605.1' '501' '503'
 '416.2' '303' '403' '106' '607.3' '502' '301' '305.1' '606.2' '203'
 '305.2' '408' '103.2' '703.1' '304' '602.1' '601.1' '706' '415' '413'
 '202.2' '305.5' '204' '105' '606.1' '410' '103.1' '406' '101' '108' '107'
 '201.1' '202.3' '409' '412' '414' '404' '407' '305.6' '705' '405' '109'
 '703.2' '607.1' '605.2' '104' '402' '505' 

Removing any empty strings in any field

In [ ]:
manifests_df = manifests_df.replace(r'^\s*$', np.nan, regex=True)
manifests_df = manifests_df.dropna()
manifests_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92041 entries, 0 to 92095
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   manifest_id  92041 non-null  object
 1   text         92041 non-null  object
 2   cmp_code     92041 non-null  object
dtypes: object(3)
memory usage: 2.8+ MB


Agregando infos de metadados dos códigos para facilitar análises

In [ ]:
coded_df = pd.merge(manifests_df, codes_df, how='left', left_on='cmp_code', right_on='code')
coded_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92041 entries, 0 to 92040
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   manifest_id     92041 non-null  object
 1   text            92041 non-null  object
 2   cmp_code        92041 non-null  object
 3   type            92041 non-null  object
 4   domain_code     92041 non-null  object
 5   domain_name     92041 non-null  object
 6   code            92041 non-null  object
 7   variable_name   92041 non-null  object
 8   title           92041 non-null  object
 9   description_md  92041 non-null  object
 10  label           92041 non-null  object
dtypes: object(11)
memory usage: 8.4+ MB


In [ ]:
missing_codes = set(foo) ^ set(list(manifests_df['cmp_code'].unique()))
print(missing_codes)
codes_df[codes_df['code'].isin(list(missing_codes))]

{'602', '305.4', '608.3', 'H'}


,type,domain_code,domain_name,code,variable_name,title,description_md,label
44,main,6,Fabric of Society,602,per602,National Way of Life: Negative,Unfavourable mentions of the manifesto country...,national way of life -
122,hb5,3,Political System,305.4,per305_4,Transition: Pre-Democratic Elites: Positive,Co-operation with pre-democratic authorities i...,transition: pre-democratic elites +
140,hb5,6,Fabric of Society,608.3,per608_3,Multiculturalism: Indigenous rights: Negative,Rejection of idea of special protection for in...,multiculturalism: indigenous rights -
144,main,0,No Domain,H,perhead,Headers,Share of headers sentences,header


In [ ]:
coded_df['domain_code'].value_counts()

5    25990
4    24618
3     9824
0     9583
7     6981
6     5570
2     5039
1     4436
Name: domain_code, dtype: int64

In [ ]:
coded_df.value_counts(['code', 'title'])
# .to_csv('/content/drive/MyDrive/UnB/TCC/dados_temporarios/manifestos_codes.csv')

code   title                                      
NA     Uncoded                                        9583
504    Welfare State Expansion                        7868
411    Technology and Infrastructure: Positive        7145
506    Education Expansion                            5254
503    Equality: Positive                             5057
                                                      ... 
703.2  Agriculture and Farmers: Negative                13
608.1  Multiculturalism General: Negative                6
305.6  Transition: Rehabilitation and Compensation       2
608.2  Multiculturalism: Immigrants Assimilation         2
102    Foreign Special Relationships: Negative           1
Length: 86, dtype: int64

Selecting columns and saving the manifesto sentences dataset

In [ ]:
coded_df = coded_df[['manifest_id','text','domain_code','domain_name','code','label']]
coded_df.to_csv('/content/drive/MyDrive/UnB/TCC/dados_processados/manifestos_sentences.csv', index=False)

## Construção da base de Manifestos


In [ ]:
sentences_df = pd.read_csv('/content/drive/MyDrive/UnB/TCC/dados_processados/manifestos_sentences.csv', keep_default_na=False)
sentences_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92041 entries, 0 to 92040
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   manifest_id  92041 non-null  object
 1   text         92041 non-null  object
 2   domain_code  92041 non-null  int64 
 3   domain_name  92041 non-null  object
 4   code         92041 non-null  object
 5   label        92041 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.2+ MB


In [ ]:
manifestos_df = sentences_df.groupby('manifest_id')['text'].apply(' '.join).reset_index()
manifestos_df

,manifest_id,text
0,180110_201010,Agenda por um Brasil Justo e Sustentável Propo...
1,180230_198911,DЕМОСRАCIА Еѕtаdо Роdеr Јudiсiáriо Соrruрçãо С...
2,180230_199410,INTRODUÇÃO O Brasil é um país viável desde que...
3,180230_199810,UNIÃO DO POVO – MUDA BRASIL Diretrizes do Prog...
4,180230_200210,"Um Brasil para Todos Crescimento, Emprego e In..."
5,180230_200610,O nome do meu segundo mandato será desenvolvim...
6,180230_201010,O Brasil mudou. E vai continuar mudando. Há qu...
7,180230_201410,"Mais Mudanças, Mais Futuro Essas são as linhas..."
8,180230_201810,PLANO DE GOVERNO 2019-2022 Coligação O POVO FE...
9,180231_200610,ÚLTIMAS NOTÍCIAS Eleições 2006 - 27/07/2006 Ma...


In [ ]:
manifestos_df.to_csv('/content/drive/MyDrive/UnB/TCC/dados_processados/manifestos_texts.csv', index=False)